# This notebook contains code used for cleaning and reducing the size of input data, for further analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
import dask.dataframe as dd

Importing both options and futures datasets in dask

In [3]:
options_df_dask = dd.read_csv("BANKNIFTY_2017_OPTIONS.csv")
options_df_dask

,Ticker,Date,Time,Open,High,Low,Close,Volume,OI,Type,Strike,Expiry,Contract_Monthly,Contract_Weekly
npartitions=9,,,,,,,,,,,,,,
,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
options_df_dask.compute()

,Ticker,Date,Time,Open,High,Low,Close,Volume,OI,Type,Strike,Expiry,Contract_Monthly,Contract_Weekly
0,BANKNIFTY25JAN1718300CE.NFO,2017-01-02,09:15:59,311.45,325.00,293.75,293.75,400.0,20040.0,CE,18300.0,2017-01-25,I,IV
1,BANKNIFTY05JAN1718900CE.NFO,2017-01-02,09:15:59,7.00,12.00,5.75,5.80,2360.0,37160.0,CE,18900.0,2017-01-05,W,I
2,BANKNIFTY05JAN1719000CE.NFO,2017-01-02,09:15:59,4.40,6.60,4.15,4.20,5680.0,120760.0,CE,19000.0,2017-01-05,W,I
3,BANKNIFTY05JAN1719100CE.NFO,2017-01-02,09:15:59,3.00,3.00,3.00,3.00,160.0,9960.0,CE,19100.0,2017-01-05,W,I
4,BANKNIFTY05JAN1719500CE.NFO,2017-01-02,09:15:59,1.50,1.50,1.50,1.50,80.0,24160.0,CE,19500.0,2017-01-05,W,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586743,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,15:29:59,95.20,99.00,93.35,93.35,23160.0,258680.0,CE,25600.0,2018-01-04,W,I
586744,BANKNIFTY04JAN1826000PE.NFO,2017-12-29,15:29:59,472.05,472.05,462.00,462.00,120.0,20520.0,PE,26000.0,2018-01-04,W,I
586745,BANKNIFTY11JAN1826000CE.NFO,2017-12-29,15:30:59,50.35,50.35,50.35,50.35,40.0,42680.0,CE,26000.0,2018-01-11,W,II
586746,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,15:30:59,99.00,99.00,99.00,99.00,40.0,258680.0,CE,25600.0,2018-01-04,W,I


In [5]:
futures_df_dask = dd.read_csv("BANKNIFTY_2017_FUTURES.csv")

Check for null/missing values

In [6]:
options_df_dask.isna().any().compute()

Ticker              False
Date                False
Time                False
Open                False
High                False
Low                 False
Close               False
Volume              False
OI                  False
Type                False
Strike              False
Expiry              False
Contract_Monthly    False
Contract_Weekly     False
dtype: bool

In [7]:
futures_df_dask.isna().any().compute()

Ticker      False
Date        False
Time        False
Open        False
High        False
Low         False
Close       False
Volume      False
OI          False
Contract    False
Expiry      False
dtype: bool

No null values in either dataset, which is good to know. Now I proceed to eliminate irrelevant data.

In [8]:
options_dask = options_df_dask.drop(['Open', 'High', 'Low', 'Volume', 'OI', 'Contract_Monthly'], axis = 1)
options_dask = options_dask[options_dask['Contract_Weekly'] == 'I']

In [9]:
futures_dask = futures_df_dask.drop(['Open', 'High', 'Low', 'Volume', 'OI'], axis = 1)
futures_dask = futures_dask[futures_dask['Contract'] == 'I']

As now all options contracts are weekly and all futures contracts are monthly, we can eliminate those columns as well

In [10]:
options_dask = options_dask.drop(['Contract_Weekly'], axis = 1)
futures_dask = futures_dask.drop(['Contract'], axis = 1)

Need to convert all timestamps to python date-time format to enable manipulation. Can round off all stamps to the next minute, as all records are at 59 seconds. Also eliminate all timestamps before 10:30 and after 15:20

In [12]:
options_dask['Time'] = dd.to_datetime(options_dask['Time'])
options_dask['Date'] = dd.to_datetime(options_dask['Date'], format='%Y-%m-%d')
options_dask['Expiry'] = dd.to_datetime(options_dask['Expiry'], format='%Y-%m-%d')

In [13]:
options_dask.compute()

,Ticker,Date,Time,Close,Type,Strike,Expiry
1,BANKNIFTY05JAN1718900CE.NFO,2017-01-02,2024-04-08 09:15:59,5.80,CE,18900.0,2017-01-05
2,BANKNIFTY05JAN1719000CE.NFO,2017-01-02,2024-04-08 09:15:59,4.20,CE,19000.0,2017-01-05
3,BANKNIFTY05JAN1719100CE.NFO,2017-01-02,2024-04-08 09:15:59,3.00,CE,19100.0,2017-01-05
4,BANKNIFTY05JAN1719500CE.NFO,2017-01-02,2024-04-08 09:15:59,1.50,CE,19500.0,2017-01-05
24,BANKNIFTY05JAN1718800CE.NFO,2017-01-02,2024-04-08 09:15:59,9.20,CE,18800.0,2017-01-05
...,...,...,...,...,...,...,...
586742,BANKNIFTY04JAN1825600PE.NFO,2017-12-29,2024-04-08 15:29:59,154.10,PE,25600.0,2018-01-04
586743,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,2024-04-08 15:29:59,93.35,CE,25600.0,2018-01-04
586744,BANKNIFTY04JAN1826000PE.NFO,2017-12-29,2024-04-08 15:29:59,462.00,PE,26000.0,2018-01-04
586746,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,2024-04-08 15:30:59,99.00,CE,25600.0,2018-01-04


Note: Above, time column contains current system date. This will be removed later and it doesnt affect the remaining computation

In [14]:
options_dask.dtypes

Ticker            object
Date      datetime64[ns]
Time      datetime64[ns]
Close            float64
Type              object
Strike           float64
Expiry    datetime64[ns]
dtype: object

As can be seen, datetime columns have been converted to python datetime datatype. Now, I will convert the dask dataframe to pandas dataframe, as dask was facing issues with manipulating datetime values.

In [15]:
options_pandas = options_dask.compute()
options_pandas

,Ticker,Date,Time,Close,Type,Strike,Expiry
1,BANKNIFTY05JAN1718900CE.NFO,2017-01-02,2024-04-08 09:15:59,5.80,CE,18900.0,2017-01-05
2,BANKNIFTY05JAN1719000CE.NFO,2017-01-02,2024-04-08 09:15:59,4.20,CE,19000.0,2017-01-05
3,BANKNIFTY05JAN1719100CE.NFO,2017-01-02,2024-04-08 09:15:59,3.00,CE,19100.0,2017-01-05
4,BANKNIFTY05JAN1719500CE.NFO,2017-01-02,2024-04-08 09:15:59,1.50,CE,19500.0,2017-01-05
24,BANKNIFTY05JAN1718800CE.NFO,2017-01-02,2024-04-08 09:15:59,9.20,CE,18800.0,2017-01-05
...,...,...,...,...,...,...,...
586742,BANKNIFTY04JAN1825600PE.NFO,2017-12-29,2024-04-08 15:29:59,154.10,PE,25600.0,2018-01-04
586743,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,2024-04-08 15:29:59,93.35,CE,25600.0,2018-01-04
586744,BANKNIFTY04JAN1826000PE.NFO,2017-12-29,2024-04-08 15:29:59,462.00,PE,26000.0,2018-01-04
586746,BANKNIFTY04JAN1825600CE.NFO,2017-12-29,2024-04-08 15:30:59,99.00,CE,25600.0,2018-01-04


In [16]:
from datetime import time
start_time = time(10, 29)
end_time = time(15, 20)
options_pandas = options_pandas[(options_pandas['Time'].dt.time >= start_time) & (options_pandas['Time'].dt.time <= end_time)]
options_pandas

,Ticker,Date,Time,Close,Type,Strike,Expiry
3819,BANKNIFTY05JAN1718800CE.NFO,2017-01-02,2024-04-08 10:29:59,3.90,CE,18800.0,2017-01-05
3836,BANKNIFTY05JAN1718700CE.NFO,2017-01-02,2024-04-08 10:29:59,6.30,CE,18700.0,2017-01-05
3838,BANKNIFTY05JAN1718600CE.NFO,2017-01-02,2024-04-08 10:29:59,10.10,CE,18600.0,2017-01-05
3839,BANKNIFTY05JAN1718000PE.NFO,2017-01-02,2024-04-08 10:29:59,118.45,PE,18000.0,2017-01-05
3840,BANKNIFTY05JAN1718400CE.NFO,2017-01-02,2024-04-08 10:29:59,25.45,CE,18400.0,2017-01-05
...,...,...,...,...,...,...,...
586059,BANKNIFTY04JAN1825200PE.NFO,2017-12-29,2024-04-08 15:19:59,35.10,PE,25200.0,2018-01-04
586060,BANKNIFTY04JAN1825300CE.NFO,2017-12-29,2024-04-08 15:19:59,258.85,CE,25300.0,2018-01-04
586061,BANKNIFTY04JAN1825300PE.NFO,2017-12-29,2024-04-08 15:19:59,55.90,PE,25300.0,2018-01-04
586062,BANKNIFTY04JAN1825400CE.NFO,2017-12-29,2024-04-08 15:19:59,192.00,CE,25400.0,2018-01-04


In [17]:
options_pandas['Time'] = pd.to_datetime(options_pandas['Time']).dt.ceil('T').dt.time
options_pandas = options_pandas.reset_index(drop=True)
options_pandas

C:\Users\thaku\AppData\Local\Temp\ipykernel_11948\1276648553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_pandas['Time'] = pd.to_datetime(options_pandas['Time']).dt.ceil('T').dt.time


,Ticker,Date,Time,Close,Type,Strike,Expiry
0,BANKNIFTY05JAN1718800CE.NFO,2017-01-02,10:30:00,3.90,CE,18800.0,2017-01-05
1,BANKNIFTY05JAN1718700CE.NFO,2017-01-02,10:30:00,6.30,CE,18700.0,2017-01-05
2,BANKNIFTY05JAN1718600CE.NFO,2017-01-02,10:30:00,10.10,CE,18600.0,2017-01-05
3,BANKNIFTY05JAN1718000PE.NFO,2017-01-02,10:30:00,118.45,PE,18000.0,2017-01-05
4,BANKNIFTY05JAN1718400CE.NFO,2017-01-02,10:30:00,25.45,CE,18400.0,2017-01-05
...,...,...,...,...,...,...,...
1942482,BANKNIFTY04JAN1825200PE.NFO,2017-12-29,15:20:00,35.10,PE,25200.0,2018-01-04
1942483,BANKNIFTY04JAN1825300CE.NFO,2017-12-29,15:20:00,258.85,CE,25300.0,2018-01-04
1942484,BANKNIFTY04JAN1825300PE.NFO,2017-12-29,15:20:00,55.90,PE,25300.0,2018-01-04
1942485,BANKNIFTY04JAN1825400CE.NFO,2017-12-29,15:20:00,192.00,CE,25400.0,2018-01-04


To futher reduce the computing power required for future analysis, I will split the data into data for call and put options. This way, generic functions can be defined for both call and put options and applied separately. We will need to consolidate the relevant data in the end, though.

In [18]:
options_pandas_CE = options_pandas[options_pandas['Type'] == 'CE']
options_pandas_PE = options_pandas[options_pandas['Type'] == 'PE']

Repeating the same procedure for futures data also, ie, convertinf to datetime datatype and keeping only relevant timestamp.

In [19]:
futures_dask['Time'] = dd.to_datetime(futures_dask['Time'])
futures_dask['Date'] = dd.to_datetime(futures_dask['Date'], format='%d-%m-%Y')
futures_dask['Expiry'] = dd.to_datetime(futures_dask['Expiry'], format='%d-%m-%Y')
futures_dask.compute()

,Ticker,Date,Time,Close,Expiry
0,BANKNIFTY-I.NFO,2017-01-02,2024-04-08 09:15:59,18213.85,2017-01-25
2,BANKNIFTY-I.NFO,2017-01-02,2024-04-08 09:16:59,18221.90,2017-01-25
6,BANKNIFTY-I.NFO,2017-01-02,2024-04-08 09:17:59,18157.30,2017-01-25
9,BANKNIFTY-I.NFO,2017-01-02,2024-04-08 09:18:59,18171.80,2017-01-25
10,BANKNIFTY-I.NFO,2017-01-02,2024-04-08 09:19:59,18159.00,2017-01-25
...,...,...,...,...,...
175515,BANKNIFTY-I.NFO,2017-12-29,2024-04-08 15:25:59,25579.15,2018-01-25
175517,BANKNIFTY-I.NFO,2017-12-29,2024-04-08 15:26:59,25579.85,2018-01-25
175519,BANKNIFTY-I.NFO,2017-12-29,2024-04-08 15:27:59,25579.00,2018-01-25
175522,BANKNIFTY-I.NFO,2017-12-29,2024-04-08 15:28:59,25580.00,2018-01-25


Note: All dates are in YYYY-MM-DD format and times are in HH:MM:SS format

In [20]:
futures_pandas = futures_dask.compute()

futures_pandas = futures_pandas[(futures_pandas['Time'].dt.time >= start_time) & (futures_pandas['Time'].dt.time <= end_time)]

futures_pandas['Time'] = pd.to_datetime(futures_pandas['Time']).dt.ceil('T').dt.time
futures_pandas = futures_pandas.reset_index(drop=True)
futures_pandas

,Ticker,Date,Time,Close,Expiry
0,BANKNIFTY-I.NFO,2017-01-02,10:30:00,18067.95,2017-01-25
1,BANKNIFTY-I.NFO,2017-01-02,10:31:00,18047.25,2017-01-25
2,BANKNIFTY-I.NFO,2017-01-02,10:32:00,18051.40,2017-01-25
3,BANKNIFTY-I.NFO,2017-01-02,10:33:00,18050.00,2017-01-25
4,BANKNIFTY-I.NFO,2017-01-02,10:34:00,18043.20,2017-01-25
...,...,...,...,...,...
71749,BANKNIFTY-I.NFO,2017-12-29,15:16:00,25562.00,2018-01-25
71750,BANKNIFTY-I.NFO,2017-12-29,15:17:00,25565.00,2018-01-25
71751,BANKNIFTY-I.NFO,2017-12-29,15:18:00,25570.00,2018-01-25
71752,BANKNIFTY-I.NFO,2017-12-29,15:19:00,25562.70,2018-01-25


In [21]:
options_pandas_CE = options_pandas_CE.reset_index(drop = True)
options_pandas_PE = options_pandas_PE.reset_index(drop = True)
options_pandas_CE

,Ticker,Date,Time,Close,Type,Strike,Expiry
0,BANKNIFTY05JAN1718800CE.NFO,2017-01-02,10:30:00,3.90,CE,18800.0,2017-01-05
1,BANKNIFTY05JAN1718700CE.NFO,2017-01-02,10:30:00,6.30,CE,18700.0,2017-01-05
2,BANKNIFTY05JAN1718600CE.NFO,2017-01-02,10:30:00,10.10,CE,18600.0,2017-01-05
3,BANKNIFTY05JAN1718400CE.NFO,2017-01-02,10:30:00,25.45,CE,18400.0,2017-01-05
4,BANKNIFTY05JAN1718500CE.NFO,2017-01-02,10:30:00,16.15,CE,18500.0,2017-01-05
...,...,...,...,...,...,...,...
915673,BANKNIFTY04JAN1826500CE.NFO,2017-12-29,15:20:00,0.95,CE,26500.0,2018-01-04
915674,BANKNIFTY04JAN1825500CE.NFO,2017-12-29,15:20:00,135.25,CE,25500.0,2018-01-04
915675,BANKNIFTY04JAN1825100CE.NFO,2017-12-29,15:20:00,418.35,CE,25100.0,2018-01-04
915676,BANKNIFTY04JAN1825300CE.NFO,2017-12-29,15:20:00,258.85,CE,25300.0,2018-01-04


In [22]:
options_pandas_PE

,Ticker,Date,Time,Close,Type,Strike,Expiry
0,BANKNIFTY05JAN1718000PE.NFO,2017-01-02,10:30:00,118.45,PE,18000.0,2017-01-05
1,BANKNIFTY05JAN1717000PE.NFO,2017-01-02,10:30:00,1.60,PE,17000.0,2017-01-05
2,BANKNIFTY05JAN1717200PE.NFO,2017-01-02,10:30:00,4.00,PE,17200.0,2017-01-05
3,BANKNIFTY05JAN1717400PE.NFO,2017-01-02,10:30:00,9.00,PE,17400.0,2017-01-05
4,BANKNIFTY05JAN1717500PE.NFO,2017-01-02,10:30:00,13.60,PE,17500.0,2017-01-05
...,...,...,...,...,...,...,...
1026804,BANKNIFTY04JAN1825000PE.NFO,2017-12-29,15:20:00,14.10,PE,25000.0,2018-01-04
1026805,BANKNIFTY04JAN1824600PE.NFO,2017-12-29,15:20:00,3.45,PE,24600.0,2018-01-04
1026806,BANKNIFTY04JAN1825200PE.NFO,2017-12-29,15:20:00,35.10,PE,25200.0,2018-01-04
1026807,BANKNIFTY04JAN1825300PE.NFO,2017-12-29,15:20:00,55.90,PE,25300.0,2018-01-04


Note: Quite a big mismatch between call options data and put options data

For every day, we only need the future price at 10.30 am. Thus, remaining timestamps can be removed, and ticker can also be removed for futures data.

In [23]:
futures_pandas = futures_pandas.drop(['Ticker'], axis = 1)
futures_pandas = futures_pandas[futures_pandas['Time'] == time(10, 30)]
futures_pandas = futures_pandas.reset_index(drop = True)
futures_pandas

,Date,Time,Close,Expiry
0,2017-01-02,10:30:00,18067.95,2017-01-25
1,2017-01-03,10:30:00,18107.00,2017-01-25
2,2017-01-04,10:30:00,18048.05,2017-01-25
3,2017-01-05,10:30:00,18079.70,2017-01-25
4,2017-01-06,10:30:00,18340.00,2017-01-25
...,...,...,...,...
241,2017-12-22,10:30:00,25640.05,2017-12-28
242,2017-12-26,10:30:00,25611.00,2017-12-28
243,2017-12-27,10:30:00,25710.05,2017-12-28
244,2017-12-28,10:30:00,25471.70,2017-12-28


In [24]:
# Combining the date and time columns into a single Datetime column
options_pandas_CE['Datetime'] = pd.to_datetime(options_pandas_CE['Date'].astype(str) + ' ' + options_pandas_CE['Time'].astype(str))
options_pandas_PE['Datetime'] = pd.to_datetime(options_pandas_PE['Date'].astype(str) + ' ' + options_pandas_PE['Time'].astype(str))

col_order = ['Ticker', 'Datetime', 'Close', 'Type', 'Strike', 'Expiry']
options_pandas_CE = options_pandas_CE.drop(['Date', 'Time'], axis = 1).reindex(columns = col_order)
options_pandas_PE = options_pandas_PE.drop(['Date', 'Time'], axis = 1).reindex(columns = col_order)


In [25]:
options_pandas_CE.to_csv('options_ce.csv', index = False)
options_pandas_PE.to_csv('options_pe.csv', index = False)
futures_pandas.to_csv('futures.csv', index = False)

Note: The below point was noticed when the overall equity value was being tracked, thus was added in the end

## Data for 10-07-2017 is not proper, caused errors later. Will remove it for options data now.

In [28]:
options_pandas_CE = options_pandas_CE[options_pandas_CE['Datetime'].dt.date != pd.to_datetime('10-07-2017', format='%d-%m-%Y').date()]
options_pandas_PE = options_pandas_PE[options_pandas_PE['Datetime'].dt.date != pd.to_datetime('10-07-2017', format='%d-%m-%Y').date()]

In [29]:
options_pandas_CE.to_csv('options_ce.csv', index = False)
options_pandas_PE.to_csv('options_pe.csv', index = False)

## These were the final, refined datasets used for implementing the strategy